# Keras Tuner

With so many potential combinations of hyperparameters to tune, coming up with the best model can be a tedious process. Often two or more parameters might have correlated effects on the overall speed of convergence as well as validation accuracy, so tuning one at a time might not lead to the best model.  And if curiosity gets the best of us, we might want to experimentation on all the hyperparameters together!  

Keras Tuner comes to automate this hyperparameter search. We define a search algorithm, the potential values that each parameter can take (e.g. discrete values or a range), our target object to maximize (e.g. validation accuracy) and sit back to see the program start training. Keras Tuner will conduct multiple experiments changing the parameters on our behalf, storing the metadata of the best model. The following code example adapted from Keras Tuner documentation showcases searching through the different model architectures (varying in the number of layers between 2 and 10) as well as varying the learning rate (between 0.1 and 0.001).

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [ ]:
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
# Input data
(x, y), (val_x, val_y) = keras.datasets.mnist.load_data()
x = x.astype('float32') / 255.
val_x = val_x.astype('float32') / 255.

In [ ]:
# Defining hyper parameters
hp = HyperParameters()
hp.Choice('learning_rate', [0.1, 0.001])
hp.Int('num_layers', 2, 10)

In [ ]:
#Defining model with expandable number of layers
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    for i in range(hp.get('num_layers')):
        model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer=keras.optimizers.Adam(hp.get('learning_rate')),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
hypermodel = RandomSearch(
    build_model,
    max_trials=20,  #Number of combinations allowed
    hyperparameters=hp,
    allow_new_entries=False,
    objective='val_accuracy')

In [ ]:
hypermodel.search(x=x, y=y, epochs=5, validation_data=(val_x, val_y))

In [ ]:
# Show summary of overall best model
hypermodel.results_summary()